# Flood Stats
- Data: Gauge, Discharge, Rainfall
- Note: Only STID RA101

## Goals
10/31 Meeting
- Clean data
- Get daily max values (creek values)
- Separate into 3 levels:
    - Minor flood stage: 8.5 ft
    - Moderate flood stage: 9.5 ft
    - Major flood stage: 11 ft
    - Minimum operating limit: 0.5 ft

## Data Dictionary
- date: date of measurement
- hour: hour of measurement
- stid: regional location
- gauge: water level in feet
- discharge: discharge level in cubic feet per second
- rainfall: rainfall level in inches
- rainfall_ft: rainfall level in feet

### Data Sources
- [Gauge and Discharge](https://waterdata.usgs.gov/monitoring-location/01464000/#parameterCode=00060&startDT=2005-10-01&endDT=2022-10-18)
- [Rainfall](https://njdep.rutgers.edu/rainfall/)

### Setup

In [102]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

In [103]:
# Display entire dataframe
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [104]:
# cd '/Users/shifraisaacs/Documents/Externship/cgi_flood_prediction_mitigation'

### Load Data

In [105]:
combined = pd.read_csv('combined_rainfall_discharge_gauge.csv', index_col=0)
print(combined.shape)
combined.head()

(131199, 7)


,agency_cd_x,guage,discharge,Date,hour,StID,rainfall
0,USGS,2.39,23.2,2007-10-01,1,RA101,0.0
1,USGS,2.38,22.6,2007-10-01,2,RA101,0.0
2,USGS,2.37,21.9,2007-10-01,3,RA101,0.0
3,USGS,2.35,20.6,2007-10-01,4,RA101,0.0
4,USGS,2.32,18.9,2007-10-01,5,RA101,0.0


### Data Cleaning and Manipulation

In [106]:
# Rearrange columns and fix spelling mistake
combined.columns = combined.columns.str.lower()
combined = combined[['date', 'stid', 'guage', 'discharge', 'rainfall']]
combined = combined.rename(columns={'guage': 'gauge', 'rainfall': 'rainfall_inches'})
combined.head()

,date,stid,gauge,discharge,rainfall_inches
0,2007-10-01,RA101,2.39,23.2,0.0
1,2007-10-01,RA101,2.38,22.6,0.0
2,2007-10-01,RA101,2.37,21.9,0.0
3,2007-10-01,RA101,2.35,20.6,0.0
4,2007-10-01,RA101,2.32,18.9,0.0


In [107]:
combined['rainfall_ft'] = round(combined['rainfall_inches']/12, 2)

In [108]:
combined['date'] = pd.to_datetime(combined['date'], infer_datetime_format=True)

In [109]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131199 entries, 0 to 131198
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   date             131199 non-null  datetime64[ns]
 1   stid             131199 non-null  object        
 2   gauge            131199 non-null  float64       
 3   discharge        131199 non-null  float64       
 4   rainfall_inches  131199 non-null  float64       
 5   rainfall_ft      131199 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 7.0+ MB


In [110]:
# Confirm no null values
combined.isnull().sum()

date               0
stid               0
gauge              0
discharge          0
rainfall_inches    0
rainfall_ft        0
dtype: int64

### Descriptive Stats

In [111]:
combined.describe()

,gauge,discharge,rainfall_inches,rainfall_ft
count,131199.000000,131199.000000,131199.000000,131199.000000
mean,3.242591,160.276956,0.005587,0.000373
std,0.758934,213.167199,0.034964,0.002962
min,2.170000,10.100000,0.000000,0.000000
25%,2.770000,58.400000,0.000000,0.000000
50%,3.080000,103.000000,0.000000,0.000000
75%,3.470000,179.000000,0.000000,0.000000
max,15.120000,5820.000000,1.700000,0.140000


### Correlational Analysis
Strong correlation between gauge and discharge; minimal correlation with rainfall

In [112]:
combined.corr()

,gauge,discharge,rainfall_inches,rainfall_ft
gauge,1.000000,0.951102,0.174789,0.159878
discharge,0.951102,1.000000,0.166233,0.152832
rainfall_inches,0.174789,0.166233,1.000000,0.967911
rainfall_ft,0.159878,0.152832,0.967911,1.000000


### Group data to get max levels for each day

In [113]:
daily_max = combined.groupby(by='date').max().reset_index()
daily_max.head()

,date,stid,gauge,discharge,rainfall_inches,rainfall_ft
0,2007-10-01,RA101,2.40,23.9,0.0,0.0
1,2007-10-02,RA101,2.39,23.2,0.0,0.0
2,2007-10-03,RA101,2.39,23.2,0.0,0.0
3,2007-10-04,RA101,2.39,23.2,0.0,0.0
4,2007-10-05,RA101,2.42,25.2,0.0,0.0


In [114]:
daily_max.describe()

,gauge,discharge,rainfall_inches,rainfall_ft
count,5477.000000,5477.000000,5477.000000,5477.000000
mean,3.464241,218.041172,0.049418,0.003851
std,0.998043,304.062274,0.126328,0.010730
min,2.320000,17.000000,0.000000,0.000000
25%,2.850000,69.300000,0.000000,0.000000
50%,3.190000,123.000000,0.000000,0.000000
75%,3.740000,242.000000,0.040000,0.000000
max,15.120000,5820.000000,1.700000,0.140000


In [115]:
daily_max.to_csv('Data/Daily_Max_Vals.csv')

## Create data subsets

In [116]:
# Minimum operating limit
op_limit = daily_max[daily_max['gauge'] >= 0.5]
print(op_limit.shape)
op_limit.head()

(5477, 6)


,date,stid,gauge,discharge,rainfall_inches,rainfall_ft
0,2007-10-01,RA101,2.40,23.9,0.0,0.0
1,2007-10-02,RA101,2.39,23.2,0.0,0.0
2,2007-10-03,RA101,2.39,23.2,0.0,0.0
3,2007-10-04,RA101,2.39,23.2,0.0,0.0
4,2007-10-05,RA101,2.42,25.2,0.0,0.0


In [117]:
# Minor flood stage
minor_filter = np.where((daily_max['gauge'] >= 8.5) & (daily_max['gauge'] < 9.5))
minor_flood = daily_max.loc[minor_filter]
print(minor_flood.shape)
minor_flood.head()

(9, 6)


,date,stid,gauge,discharge,rainfall_inches,rainfall_ft
667,2009-08-02,RA101,8.93,1820.0,1.10,0.09
795,2009-12-09,RA101,8.82,1990.0,0.51,0.04
888,2010-03-13,RA101,9.36,2450.0,0.34,0.03
889,2010-03-14,RA101,9.35,2440.0,0.28,0.02
905,2010-03-30,RA101,8.98,2260.0,0.30,0.02


In [118]:
# Moderate flood stage
moderate_filter = np.where((daily_max['gauge'] >= 9.5) & (daily_max['gauge'] < 11))
moderate_flood = daily_max.loc[moderate_filter]
print(moderate_flood.shape)
moderate_flood.head()

(9, 6)


,date,stid,gauge,discharge,rainfall_inches,rainfall_ft
687,2009-08-22,RA101,9.74,2170.0,1.70,0.14
688,2009-08-23,RA101,10.63,2560.0,0.02,0.00
1287,2011-04-17,RA101,9.63,2620.0,0.38,0.03
1422,2011-08-30,RA101,10.10,2870.0,0.00,0.00
2387,2014-04-30,RA101,10.96,3350.0,0.48,0.04


In [119]:
# Major flood stage
major_filter = np.where(daily_max['gauge'] > 11)
major_flood = daily_max.loc[major_filter]
print(major_flood.shape)
major_flood.head()

(4, 6)


,date,stid,gauge,discharge,rainfall_inches,rainfall_ft
1420,2011-08-28,RA101,15.12,5820.0,0.60,0.05
1421,2011-08-29,RA101,14.99,5730.0,0.00,0.00
2388,2014-05-01,RA101,11.62,3710.0,0.21,0.02
5061,2021-09-02,RA101,12.59,4260.0,0.13,0.01
